In [ ]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
!pip install xlsxwriter
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 22.6 MB/s 


In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [ ]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/stat?token={IEX_CLOUD_API_TOKEN}'
data= requests.get(api_url).json()
data



{'avg10Volume': 89903919,
 'avg30Volume': 110480213,
 'beta': 1.3392058989439346,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 171.85,
 'day30ChangePercent': -0.12406609512545874,
 'day50MovingAvg': 154.16,
 'day5ChangePercent': -0.02976922846862609,
 'dividendYield': 0.006226235479587191,
 'employees': 152145,
 'exDividendDate': '2022-05-06',
 'float': 0,
 'marketcap': 2362134034911,
 'maxChangePercent': 55.37907780013188,
 'month1ChangePercent': -0.08896230157317134,
 'month3ChangePercent': -0.12857890187866525,
 'month6ChangePercent': -0.10393332553768943,
 'nextDividendDate': '',
 'nextEarningsDate': '2022-07-19',
 'peRatio': 24.062549869836534,
 'sharesOutstanding': 16388346615,
 'ttmDividendRate': 0.9297552529727252,
 'ttmEPS': 6.44,
 'week52change': 0.16938537016278657,
 'week52high': 189.71,
 'week52highSplitAdjustOnly': 190.26,
 'week52low': 128.01,
 'week52lowSplitAdjustOnly': 127.89,
 'year1ChangePercent': 0.18602575322339293,
 'year2ChangePercent': 0.8130565487512702,
 

In [ ]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

my_columns = ['Ticker',"Company's Name", 'Price', 'One-Year Price Return', 'Number of Shares to Buy']



In [ ]:
def portfolio_input():
  global portfolio_size
  portfolio_size=input("Enter the size of the portfolio: ")
  try:
    float(portfolio_size)
  except ValueError:
    print("That's not a number! Enter an integer value eg:- 100000,100,555 :")
    portfolio_size=input("Enter the size of the portfolio: ")

portfolio_input()

Enter the size of the portfolio: 100000


In [ ]:
hqm_columns = [
                'Ticker', 
                'Price', 
               "Company's Name",
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
  batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    if symbol == 'DISCA' or symbol == 'HFC' or symbol == 'VIAC' or symbol=='WLTW':
           continue
    hqm_dataframe = hqm_dataframe.append(
                                    pd.Series([symbol, 
                                                data[symbol]['quote']['latestPrice'],
                                                data[symbol]['quote']['companyName'],
                                                'N/A',
                                                data[symbol]['stats']['year1ChangePercent'],
                                                'N/A',
                                                data[symbol]['stats']['month6ChangePercent'],
                                                'N/A',
                                                data[symbol]['stats']['month3ChangePercent'],
                                                'N/A',
                                                data[symbol]['stats']['month1ChangePercent'],
                                                'N/A',
                                                'N/A'
                                                ], 
                                              index = hqm_columns), 
                                    ignore_index = True)
        
hqm_dataframe.columns



Index(['Ticker', 'Price', 'Company's Name', 'Number of Shares to Buy',
       'One-Year Price Return', 'One-Year Return Percentile',
       'Six-Month Price Return', 'Six-Month Return Percentile',
       'Three-Month Price Return', 'Three-Month Return Percentile',
       'One-Month Price Return', 'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [ ]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]
for row in hqm_dataframe.index:
    for time_period in time_periods:
      if hqm_dataframe.loc[row,f'{time_period} Price Return'] == None:
        hqm_dataframe.loc[row,f'{time_period} Price Return']=0
for row in hqm_dataframe.index:
    for time_period in time_periods:
      hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row,f'{time_period} Price Return'])
hqm_dataframe

,Ticker,Price,Company's Name,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,128.76,Agilent Technologies Inc.,N/A,-0.053429,45.508982,-0.142007,30.538922,-0.063459,41.317365,0.05123,81.636727,N/A
1,AAL,16.59,American Airlines Group Inc,N/A,-0.357639,5.189621,-0.024361,53.093812,0.03282,71.257485,-0.158128,3.792415,N/A
2,AAP,192.35,Advance Auto Parts Inc,N/A,0.019574,60.279441,-0.156499,27.345309,-0.080984,37.52495,-0.046306,23.952096,N/A
3,AAPL,149.28,Apple Inc,N/A,0.192162,83.632735,-0.099307,40.319361,-0.128033,22.954092,-0.089058,12.37525,N/A
4,ABBV,154.46,Abbvie Inc,N/A,0.376339,93.213573,0.274168,90.01996,-0.013648,56.087824,-0.016999,40.718563,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,123.22,Yum Brands Inc.,N/A,0.017329,58.882236,-0.040692,51.497006,-0.004206,58.483034,0.037731,75.848303,N/A
497,ZBH,123.48,Zimmer Biomet Holdings Inc,N/A,-0.24389,14.171657,-0.002965,56.287425,-0.01053,56.886228,-0.000427,50.299401,N/A
498,ZBRA,341.49,Zebra Technologies Corp. - Class A,N/A,-0.347965,5.988024,-0.44934,1.197605,-0.182104,10.978044,-0.091818,11.576846,N/A
499,ZION,58.87,Zions Bancorporation N.A,N/A,-0.030874,49.500998,-0.089888,42.51497,-0.171814,13.173653,-0.01674,41.117764,N/A


In [ ]:
from statistics import mean
for row in hqm_dataframe.index:
  momentum_percentiles=[]
  for time_period in time_periods:
    momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
  hqm_dataframe.loc[row, 'HQM Score']= mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Company's Name,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,128.76,Agilent Technologies Inc.,N/A,-0.053429,45.508982,-0.142007,30.538922,-0.063459,41.317365,0.05123,81.636727,49.750499
1,AAL,16.59,American Airlines Group Inc,N/A,-0.357639,5.189621,-0.024361,53.093812,0.03282,71.257485,-0.158128,3.792415,33.333333
2,AAP,192.35,Advance Auto Parts Inc,N/A,0.019574,60.279441,-0.156499,27.345309,-0.080984,37.52495,-0.046306,23.952096,37.275449
3,AAPL,149.28,Apple Inc,N/A,0.192162,83.632735,-0.099307,40.319361,-0.128033,22.954092,-0.089058,12.37525,39.820359
4,ABBV,154.46,Abbvie Inc,N/A,0.376339,93.213573,0.274168,90.01996,-0.013648,56.087824,-0.016999,40.718563,70.00998
...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,123.22,Yum Brands Inc.,N/A,0.017329,58.882236,-0.040692,51.497006,-0.004206,58.483034,0.037731,75.848303,61.177645
497,ZBH,123.48,Zimmer Biomet Holdings Inc,N/A,-0.24389,14.171657,-0.002965,56.287425,-0.01053,56.886228,-0.000427,50.299401,44.411178
498,ZBRA,341.49,Zebra Technologies Corp. - Class A,N/A,-0.347965,5.988024,-0.44934,1.197605,-0.182104,10.978044,-0.091818,11.576846,7.43513
499,ZION,58.87,Zions Bancorporation N.A,N/A,-0.030874,49.500998,-0.089888,42.51497,-0.171814,13.173653,-0.01674,41.117764,36.576846


In [ ]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe= hqm_dataframe[:50]
hqm_dataframe.reset_index( inplace=True, drop= True)
hqm_dataframe


,Ticker,Price,Company's Name,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MRO,32.75,Marathon Oil Corporation,N/A,1.348842,99.401198,1.13027,99.800399,0.388593,99.001996,0.218915,99.201597,99.351297
1,OXY,71.65,Occidental Petroleum Corp.,N/A,1.470691,99.600798,1.449497,100.0,0.494592,99.800399,0.19375,98.003992,99.351297
2,DVN,77.85,Devon Energy Corp.,N/A,1.637731,99.800399,0.952431,99.401198,0.347191,98.403194,0.202787,98.403194,99.001996
3,VLO,135.68,Valero Energy Corp.,N/A,0.723748,97.60479,1.006088,99.600798,0.628069,100.0,0.128599,94.610778,97.954092
4,APA,50.10,APA Corporation,N/A,1.144131,99.201597,0.943731,99.001996,0.274398,97.804391,0.140749,95.409182,97.854291
5,MPC,109.61,Marathon Petroleum Corp,N/A,0.754859,98.203593,0.751158,98.003992,0.390252,99.201597,0.149132,96.007984,97.854291
6,COP,121.50,Conoco Phillips,N/A,1.066069,99.001996,0.697734,97.60479,0.227057,96.007984,0.207001,98.802395,97.854291
7,HRB,36.81,H&R Block Inc.,N/A,0.522325,95.608782,0.558636,96.007984,0.401744,99.401198,0.401919,100.0,97.754491
8,EOG,149.57,"EOG Resources, Inc.",N/A,0.747326,98.003992,0.751716,98.203593,0.254136,97.005988,0.183408,97.40519,97.654691
9,HAL,42.62,Halliburton Co.,N/A,0.714915,97.40519,0.945918,99.201597,0.260205,97.205589,0.145615,95.808383,97.40519


In [ ]:
position_size= float(portfolio_size)/len(hqm_dataframe.index)
for row in hqm_dataframe.index:
  hqm_dataframe.loc[row, 'Number of Shares to Buy']= math.floor(position_size/hqm_dataframe.loc[row,'Price'])
hqm_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Company's Name,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MRO,32.75,Marathon Oil Corporation,61,1.348842,99.401198,1.13027,99.800399,0.388593,99.001996,0.218915,99.201597,99.351297
1,OXY,71.65,Occidental Petroleum Corp.,27,1.470691,99.600798,1.449497,100.0,0.494592,99.800399,0.19375,98.003992,99.351297
2,DVN,77.85,Devon Energy Corp.,25,1.637731,99.800399,0.952431,99.401198,0.347191,98.403194,0.202787,98.403194,99.001996
3,VLO,135.68,Valero Energy Corp.,14,0.723748,97.60479,1.006088,99.600798,0.628069,100.0,0.128599,94.610778,97.954092
4,APA,50.10,APA Corporation,39,1.144131,99.201597,0.943731,99.001996,0.274398,97.804391,0.140749,95.409182,97.854291
5,MPC,109.61,Marathon Petroleum Corp,18,0.754859,98.203593,0.751158,98.003992,0.390252,99.201597,0.149132,96.007984,97.854291
6,COP,121.50,Conoco Phillips,16,1.066069,99.001996,0.697734,97.60479,0.227057,96.007984,0.207001,98.802395,97.854291
7,HRB,36.81,H&R Block Inc.,54,0.522325,95.608782,0.558636,96.007984,0.401744,99.401198,0.401919,100.0,97.754491
8,EOG,149.57,"EOG Resources, Inc.",13,0.747326,98.003992,0.751716,98.203593,0.254136,97.005988,0.183408,97.40519,97.654691
9,HAL,42.62,Halliburton Co.,46,0.714915,97.40519,0.945918,99.201597,0.260205,97.205589,0.145615,95.808383,97.40519


In [ ]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)
writer.save()

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ["Company's Name", string_template],
                    'D': ['Number of Shares to Buy', integer_template],
                    'E': ['One-Year Price Return', percent_template],
                    'F': ['One-Year Return Percentile', percent_template],
                    'G': ['Six-Month Price Return', percent_template],
                    'H': ['Six-Month Return Percentile', percent_template],
                    'I': ['Three-Month Price Return', percent_template],
                    'J': ['Three-Month Return Percentile', percent_template],
                    'K': ['One-Month Price Return', percent_template],
                    'L': ['One-Month Return Percentile', percent_template],
                    'M': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)


In [ ]:
writer.save()

/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
